<a href="https://colab.research.google.com/github/ritam006/Sentence-Classification/blob/master/CNN_for_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q gensim

In [0]:
import gensim
import os
from google.colab import drive
import numpy as np

In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.getcwd()

'/content'

In [0]:
os.chdir('drive/My Drive')

In [0]:
os.listdir()

['T&P',
 'Project Machine learning',
 'Fable ',
 'Discover Personalization.gsheet',
 'Nainitaal 2017',
 'Idea.gdoc',
 'Untitled document.gdoc',
 'Shimla Manali 2018',
 'NLP Project',
 'NLP Major']

In [0]:
os.chdir('NLP Project')

In [0]:
os.listdir()

['data',
 'Abstract.gdoc',
 'GoogleNews-vectors-negative.bin',
 'glove.6B',
 'rnn_model.h5',
 'rnn_model.yaml',
 'sentiment analysisusing RNN train.ipynb',
 'cnn_glove_300d.h5',
 'cnn_glove_300d.yaml',
 'cnn_model_predict.ipynb',
 'CNN for text classification.ipynb']

In [0]:
def load_glove(embedding_dim):
  path = os.path.join(os.getcwd(),'glove.6B/glove.6B.{}d.txt'.format(embedding_dim))
  f = open(path,'r')
  embedding_weight = {}  
  for line in f:
    values = line.split()
    word = values[0]
    coeff = np.asarray(values[1:],dtype='float32')
    embedding_weight[word] = coeff    
      
  return embedding_weight   

In [0]:
def load_word2vec(word2id):  
  id2word = {i:word for word,i in word2id.items()}
  models = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative.bin',binary=True)  
  embedding_weights = {i: models[word] if word in models else
                              np.random.uniform(-0.25, 0.25, models.vector_size)
                         for i, word in id2word.items()}
  return embedding_weights

In [0]:
# Model Hyperparameters
embedding_dim = 300
filter_sizes = (3, 4, 5)
num_filters = 100
dropout_prob = (0.5, 0.5)
hidden_dims = 50

# Training parameters
batch_size = 64
num_epochs = 10

# Prepossessing parameters
sequence_length = 400
max_words = 5000

In [0]:
from keras.datasets import imdb
from keras.preprocessing import sequence

Using TensorFlow backend.


In [0]:
def load_data(max_words,sequence_length):
  (x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=max_words , )
  x_train = sequence.pad_sequences(x_train, maxlen = sequence_length, padding = 'post',truncating='post')
  x_test = sequence.pad_sequences(x_test, maxlen=sequence_length, padding = 'post', truncating = 'post')
  word2id = imdb.get_word_index()
  return x_train,y_train,x_test,y_test,word2id

In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate
from keras.initializers import glorot_uniform
from keras.constraints import max_norm

In [0]:
x_train,y_train,x_test,y_test,word2id=load_data(max_words,sequence_length)
id2word = {i:word for word,i in word2id.items()}


1646592/1641221 [==============================] - 0s 0us/step


In [0]:
#embedding_weights=load_word2vec(id2word)
embedding_weight = load_glove(embedding_dim)
embedding_matrix = np.zeros((len(word2id)+1,embedding_dim))
count = 0
for word,i in word2id.items():
  wordvec = embedding_weight.get(word)
  if wordvec is None:
    wordvec = np.random.uniform(-0.25,0.25,embedding_dim)
    count = count+1
   
  embedding_matrix[i]=wordvec

print('{} words were not in the glove'.format(count))

28434 words were not in the glove


In [0]:
print(count)
len(embedding_weight)
embedding_matrix.shape

28434


(88585, 300)

In [0]:
print(len(id2word))
print(x_train.shape)
print(x_test.shape)
print('Vocabulary size :{:d}'.format(len(id2word)))

88584
(25000, 400)
(25000, 400)
Vocabulary size :88584


In [0]:
def load_model(sequence_length,embedding_dim,id2word,dropout_prob,filter_sizes,num_filters):
  model_input = Input(shape = (sequence_length,))
  x1 = Embedding(len(id2word)+1,embedding_dim, weights = [embedding_matrix], input_length = sequence_length , trainable = True)(model_input)
  #x1 = Dropout(dropout_prob[0])(x1)
  
  x2 = Embedding(len(id2word)+1,embedding_dim, embeddings_initializer = 'glorot_uniform' , input_length = sequence_length , trainable = True)(model_input)
  #x2 = Dropout(dropout_prob[0])(x2)
  
  conv_blocks = []
  for sz in filter_sizes:
    conv = Convolution1D(filters = num_filters,kernel_size=sz,padding='valid',activation='relu',strides=1)(x1)
    conv = MaxPooling1D(pool_size=2)(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)
    
  for sz in filter_sizes:
    conv = Convolution1D(filters = num_filters,kernel_size=sz,padding='valid',activation='relu',strides=1)(x2)
    conv = MaxPooling1D(pool_size=2)(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)
    
  x=Concatenate()(conv_blocks)
  x=Dropout(dropout_prob[1])(x)
  x=Dense(hidden_dims,kernel_constraint = max_norm(3.),activation='relu')(x)
  model_output = Dense(1,activation='sigmoid')(x)
  
  model = Model(model_input,model_output)  
  return model  

In [0]:
model = load_model(sequence_length,embedding_dim,id2word,dropout_prob,filter_sizes,num_filters)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
model.fit(x_train,y_train,batch_size=batch_size,epochs=num_epochs,verbose=1,validation_data=(x_test,y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 85s 3ms/step - loss: 0.4227 - acc: 0.7876 - val_loss: 0.2882 - val_acc: 0.8801
Epoch 2/10
25000/25000 [==============================] - 80s 3ms/step - loss: 0.2380 - acc: 0.9057 - val_loss: 0.2882 - val_acc: 0.8770
Epoch 3/10
25000/25000 [==============================] - 80s 3ms/step - loss: 0.2057 - acc: 0.9202 - val_loss: 0.3178 - val_acc: 0.8692
Epoch 4/10
25000/25000 [==============================] - 80s 3ms/step - loss: 0.1725 - acc: 0.9348 - val_loss: 0.3126 - val_acc: 0.8729
Epoch 5/10
25000/25000 [==============================] - 80s 3ms/step - loss: 0.1506 - acc: 0.9441 - val_loss: 0.3232 - val_acc: 0.8676
Epoch 6/10
25000/25000 [==============================] - 80s 3ms/step - loss: 0.1302 - acc: 0.9525 - val_loss: 0.3779 - val_acc: 0.8650
Epoch 7/10
25000/25000 [==============================] - 80s 3ms/step - loss: 0.1138 - acc: 0.9578 - val_loss: 0.3849 - val_acc

In [0]:
model.save_weights('cnn_multichannel_glove.h5')

In [0]:
model_yaml = model.to_yaml()
with open('cnn_multichannel_glove.yaml','w')as yaml_file:
  yaml_file.write(model_yaml)

yaml_file.close()
if 'cnn_glove_300d.yaml' in os.listdir():
  print('Saved model !!')

Saved model !!


In [0]:
import pickle

In [0]:
with open('word_index.pkl','wb') as f:
  pickle.dump(word2id,f)
print('Saved the word to index list !!')

Saved the word to index list !!


In [0]:
from google.colab import files

In [0]:
files.download('word_index.pkl')

In [0]:
y_train.shape

(25000,)

In [0]:
y_train[:50]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0])